# Clustering over INCOME and AGE across the IRIS

# Import

In [ ]:
%pip install openpyxl   # for pd.read_excel
%pip install ipywidgets # for tqdm.notebook
%pip install pot        # for ot

In [ ]:
import numpy as np
import ot    # pip install pot
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import geopandas as gpd

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import sys
sys.path.append("../src")
from clustering_methods import bary_WKMeans, dist_WKMeans
from utils import reconstruct_joint_distribution_ot, create_regular_grid, project_distribution_on_grid, computeDistanceMatrix, plot_projected_distribution


### Income Deciles per IRIS

In [ ]:
# Load the file
df_income_temp = pd.read_csv('../data/BASE_TD_FILO_DEC_IRIS_2020.csv', sep=';', decimal=',')


# Decile columns
decile_cols = ['DEC_D120', 'DEC_D220', 'DEC_D320', 'DEC_D420', 'DEC_MED20', 'DEC_D620', 'DEC_D720', 'DEC_D820', 'DEC_D920']

# Convert from object to float64
df_income_temp[decile_cols] = df_income_temp[decile_cols].apply(pd.to_numeric, errors='coerce')

# Remove the 1319 rows with missing deciles
df_income = df_income_temp.dropna(subset=decile_cols).reset_index(drop=True)

distributions = df_income[decile_cols].to_numpy()

# Useful 
colors = ['cornflowerblue', 'forestgreen', 'red', 'deeppink', 'orange', 'brown', 'purple']
age_groups = ["0_17", "18_29", "30_39", "40_49", "50_64", "65_74", "75P"]


n_distributions = distributions.shape[0]

### Average Income per age

In [ ]:
# Load the file
df_income_age = pd.read_excel("../data/reve-niv-vie-individu-age-med.xlsx", skiprows=3)

# Keep only the 2020 column
df_income_age = df_income_age[["Tranche d’âge", '2020³ ⁴']]

# Rename the column
df_income_age = df_income_age.rename(columns={'2020³ ⁴': "MEDIAN_INCOME"})

# Rename the age groups
renaming_dict = {
    "Moins de 18 ans": "0_17",
    "De 18 à 29 ans": "18_29",
    "De 30 à 39 ans": "30_39",
    "De 40 à 49 ans": "40_49",
    "De 50 à 64 ans": "50_64",
    "De 65 à 74 ans": "65_74",
    "75 ans et plus": "75P"
}
df_income_age["AGE_GROUP"] = df_income_age["Tranche d’âge"].replace(renaming_dict)

# Get rid of the metadata lines
df_income_age = df_income_age[df_income_age['AGE_GROUP'].isin(age_groups)].reset_index(drop=True)

# Keep only the relevant columns
df_income_age = df_income_age[["AGE_GROUP", "MEDIAN_INCOME"]]


In [ ]:
df_income.head()

### Population per age 

To apply our algorithms using income by age group, we need to estimate the population in each corresponding age range for every IRIS. However, the INSEE population dataset provides age groupings that do not match exactly with those used in the national income-by-age statistics. To reconcile the two, we reconstruct the target age groups — (0–17), (18–29), (30–39), (40–49), (50–64), (65–74), and 75+ — by combining and subtracting available groups from the population dataset. In some cases, we rely on approximations (e.g., estimating the 45–54 group to split 40–49 and 50–64) to ensure consistency with the income data structure. This harmonization step is essential to later compute joint distributions of age and income at the IRIS level.

In [ ]:
# Load the Excel sheet
df_pop_temp = pd.read_excel("../data/base-ic-evol-struct-pop-2020.xlsx", sheet_name=0, skiprows=5)

# Directly accessible age groups
df_pop_temp["AGE_0_17"] = df_pop_temp["P20_POP0002"] + df_pop_temp["P20_POP0305"] + df_pop_temp["P20_POP0610"] + df_pop_temp["P20_POP1117"]
df_pop_temp["AGE_18_29"] = df_pop_temp["P20_POP0014"] + df_pop_temp["P20_POP1529"] - df_pop_temp["AGE_0_17"]
df_pop_temp["AGE_30_39"] = df_pop_temp["P20_POP2539"] + df_pop_temp["P20_POP1824"] - df_pop_temp["AGE_18_29"]
df_pop_temp["AGE_65_74"] = df_pop_temp["P20_POP65P"] - df_pop_temp["P20_POP75P"]
# No processing needed for 75+ age group

# Approximation of the 45–54 group to help deduce 40–49 and 50–64
df_pop_temp["AGE_45_54"] = (
    df_pop_temp["AGE_0_17"] + df_pop_temp["P20_POP1824"] + df_pop_temp["P20_POP2539"] + df_pop_temp["P20_POP4054"]
    - (df_pop_temp["P20_POP0014"] + df_pop_temp["P20_POP1529"] + df_pop_temp["P20_POP3044"])
)

# Approximate 40–49 from 40–54 and part of 45–54
df_pop_temp["AGE_40_49"] = df_pop_temp["P20_POP4054"] - 0.5 * df_pop_temp["AGE_45_54"]

# Approximate 50–64 from 55–64 and part of 45–54
df_pop_temp["AGE_50_64"] = df_pop_temp["P20_POP5564"] + 0.5 * df_pop_temp["AGE_45_54"]

# Renaming for clarity
df_pop_temp["0_17"] = df_pop_temp["AGE_0_17"]
df_pop_temp["18_29"] = df_pop_temp["AGE_18_29"]
df_pop_temp["30_39"] = df_pop_temp["AGE_30_39"]
df_pop_temp["40_49"] = df_pop_temp["AGE_40_49"]
df_pop_temp["50_64"] = df_pop_temp["AGE_50_64"]
df_pop_temp["65_74"] = df_pop_temp["AGE_65_74"]
df_pop_temp["75P"] = df_pop_temp["P20_POP75P"]

# Final columns to keep
final_cols = ["IRIS", "REG", "DEP", "LIBCOM", "LIBIRIS"] + age_groups

df_pop = df_pop_temp[final_cols]


## Joint Age-Income Distributions

### Reconstructing the joint age–income distribution

For each IRIS, we reconstruct a joint probability distribution π over age groups and income levels. The marginal distributions are:
* the age distribution of the IRIS (based on population counts per age group), and
* a uniform distribution over the income deciles observed in the IRIS.

We use optimal transport to find the coupling π that best aligns the income levels with the national average income for each age group, minimizing the cost:

$$
C_{i,j} = |R_j - m(T_i)|
$$

where $R_j$ is the j-th income level (decile), and $m(T_i)$ is the national average income for age group $T_i$.

The result is a joint distribution π (a matrix), interpreted as the most plausible alignment between age and income in the IRIS, under minimal deviation from national trends.

### Computation for Each IRIS

In [ ]:
# Compute national median income per age group
age_income_medians = df_income_age.set_index("AGE_GROUP").loc[age_groups, "MEDIAN_INCOME"].to_numpy()

# Find common IRIS identifiers present in both datasets
common_iris = sorted(set(df_income["IRIS"]).intersection(df_pop["IRIS"]))

# Compute joint distributions
joint_distributions = []
joint_supports = []
iris_ids = []

for iris in tqdm(common_iris):
    row_income = df_income[df_income["IRIS"] == iris]
    row_pop = df_pop[df_pop["IRIS"] == iris]

    if row_income.empty or row_pop.empty:
        continue  # Skip if data is missing for this IRIS

    # Age group weights (normalized)
    age_counts = row_pop.iloc[0][age_groups].to_numpy()
    age_weights = age_counts / age_counts.sum()

    # Income decile values
    income_deciles = row_income.iloc[0][decile_cols].to_numpy()

    # Compute the joint distribution π using optimal transport
    pi, supp= reconstruct_joint_distribution_ot(age_weights, income_deciles, age_income_medians)

    joint_supports.append(supp)
    joint_distributions.append(pi)
    iris_ids.append(iris)

# Normalisation des supports pour équilibrer les échelles âge/revenu
def normalize_supports(supports):
    all_supports = np.vstack(supports)
    min_vals = all_supports.min(axis=0)
    max_vals = all_supports.max(axis=0)
    normalized = [(s - min_vals) / (max_vals - min_vals) for s in supports]
    return normalized, min_vals, max_vals

joint_supports, support_min, support_max = normalize_supports(joint_supports)


# Stack the joint distributions into a 3D array: (n_iris, n_age_groups, n_income_deciles)
joint_supports = np.stack(joint_supports)
joint_distributions = np.stack(joint_distributions)


In [ ]:
# Grid creation
grid = create_regular_grid(n_bins_age=20, n_bins_income=20)

# Projection
projected_distributions = np.array([
    project_distribution_on_grid(supp.reshape(-1, 2), dist.flatten(), grid)
    for supp, dist in zip(joint_supports, joint_distributions)
])


In [ ]:
print(support_max)

In [ ]:
i = np.random.default_rng(42)

plot_projected_distribution(
    projected_distributions[i],
    grid_shape=(20, 20),
    support_min=support_min,
    support_max=support_max
)


# Clustering

## Introduction to Wasserstein K-means

### Distance in the Wasserstein space

We will now try to cluster our IRIS based on the joint-distributions we computed earlier.
To apply a K-means algorithm to this data, we must first define a meaningful distance between observations.

We choose the Earth Mover's Distance (EMD), which corresponds to the Wasserstein distance of order 1. It arises from the Kantorovich optimal transport problem:

$$
\min_{\pi\in\mathbb R_+^{n\times m}}
\sum_{i=1}^n\sum_{j=1}^m \pi_{ij}\,M_{ij}
\quad\text{such that}\quad 
\sum_j \pi_{ij} = a_i,\quad
\sum_i \pi_{ij} = b_j.
$$

Here:

* The constraint $\sum_j \pi_{ij}=a_i$ ensures that the **first marginal** of $\pi$ is $a$.
* The constraint $\sum_i \pi_{ij}=b_j$ ensures that the **second marginal** of $\pi$ is $b$.
* The result of the optimization, called the **cost**, represents the **minimal total cost** of transporting mass from $a$ to $b$ under cost matrix $M$.

This distance is easily computable using the [POT library](https://pythonot.github.io/) (Python Optimal Transport), via the function:

```python
cost = ot.emd2(a, b, M)
```

Where:

* **`a`** is a probability vector of size $n$: $a_i \geq 0$, $\sum_i a_i = 1$. It represents the **first distribution**, supported on $n$ points.
* **`b`** is a probability vector of size $m$: $b_j \geq 0$, $\sum_j b_j = 1$. It represents the **second distribution**, supported on $m$ points.
* **`M`** is a cost matrix of shape $n \times m$, where each element $M_{ij}$ represents the **cost of transporting one unit of mass** from location $i$ to location $j$.

### Quick addition for the cost matrix

All of our joint_distributions share a common discrete structure (7 age groups × 9 income deciles), because we index revenue not by its actual value but by its ordinal decile position. Concretely, each “bin” corresponds to a pair $(\text{age\_idx}, \text{decile\_idx})$, forming a fixed grid.

Because this grid is identical for every IRIS, we can use a single cost matrix for all comparisons:

```python
coords = [(i, j) for i in range(7) for j in range(9)]
cost_matrix = ot.dist(coords, coords)
```

This matrix defines the ground cost as the Euclidean distance between bins in the integer index space. This shared metric lets us compute meaningful Wasserstein distances across different IRIS efficiently, without needing to recompute it for each pair. As noted in optimal transport theory, the cost matrix is independent of the actual distributions because it only depends on how we define the “bins” themselves.

## Clustering based on barycenters (B-WKM)

This **centroid-based Wasserstein K-means** extends the classical Lloyd’s K-means to the space of probability measures by replacing Euclidean centroids and norms with **Wasserstein barycenters** and $W_2^2$ distances. As outlined by Domazakis et al. (2019) and formalized by Zhuang et al. (2022), the algorithm iteratively alternates between assignment and update (see below).

In [ ]:
data = joint_distributions[:1000]
n_clusters = 2
random_state = 42
reg=1e-1

rng = np.random.default_rng(random_state)
n_samples, n_bins = data.shape

cost_matrix = ot.dist(grid, grid)


In [ ]:
# Assignment step
distances = np.array([
    [ot.emd2(data[i], barycenters[k], cost_matrix) for k in range(n_clusters)]
    for i in range(n_samples)
])
new_assignments = distances.argmin(axis=1)


# Update step
new_barycenters = []
for k in range(n_clusters):
    indices_k = np.where(assignments == k)[0]
    if len(indices_k) == 0:
        new_barycenters.append(data[rng.integers(0, n_samples)])
    else:
        cluster_hists = data[indices_k].T
        bary = ot.bregman.barycenter(cluster_hists, cost_matrix, reg)
        new_barycenters.append(bary)

barycenters = new_barycenters

return assignments, barycenters

* **Assignment step**: at each iteration, assign each distribution $\mu_i$ to the cluster whose centroid (a Wasserstein barycenter $\nu_k$) minimizes $W_2(\mu_i, \nu_k)$.

In [ ]:
# Normalize the input distributions
data = data / data.sum(axis=1, keepdims=True)

# Initialisation of the barycenters
indices = rng.choice(n_samples, size=n_clusters, replace=False)
barycenters = data[indices].copy()

assignments = np.full(n_samples, -1)

for i in tqdm(range(n_samples), desc="Assigning to barycenters"):
    distances = np.zeros(n_clusters, dtype=float)
    for j in range(n_clusters):
        # Compute cost matrix
        cost_matrix = ot.dist(supports[i], supports[j])
        distances[j] = ot.emd2(data_flat[i], bary_flat[j], cost_matrix)
    assignments[i] = np.argmin(distances)

In [ ]:
# Normalize the input distributions
data = data / data.sum(axis=1, keepdims=True)

# Initialisation of the barycenters
indices = rng.choice(n_samples, size=n_clusters, replace=False)
barycenters = data[indices].copy()

assignments = np.full(n_samples, -1)

# Assignment step
distances = np.array([
    [ot.emd2(data[i], barycenters[k], cost_matrix) for k in range(n_clusters)]
    for i in range(n_samples)
])
assignments = distances.argmin(axis=1)

* **Centroid update step**: recompute each cluster’s centroid as the Wasserstein barycenter of all $\mu_i$ in that cluster—i.e., solve 

$$
\nu_k = \arg \min_{\nu} \frac 1 {|G_k|} \sum_{i \in G_k} W_2^2(\mu_i,\nu)
$$


In [ ]:
bary_flat = []
bary_supports = []

for k in tqdm(range(n_clusters), desc="Recomputing barycenters"):
    # Indices des distributions du cluster k
    cluster_indices = np.where(assignments == k)[0]

    # If the cluster is empty, reinitialize with a random sample
    if len(cluster_indices) == 0:
        idx = rng.integers(0, data.shape[0])
        bary_flat.append(data_flat[idx])
        bary_supports.append(supports[idx])
        continue

    # Récupérer les histogrammes et les supports associés
    cluster_hists = [data_flat[i] for i in cluster_indices]
    cluster_supports = [supports[i] for i in cluster_indices]

    # Construire le support commun (union des supports)
    combined_support = np.unique(np.vstack(cluster_supports), axis=0)

    # Reprojeter chaque histogramme sur le support commun
    projected_hists = []
    for hist, supp in zip(cluster_hists, cluster_supports):
        proj = np.zeros(len(combined_support))
        for i, atom in enumerate(supp):
            # Trouver l’indice correspondant dans le support commun
            idx = np.where((combined_support == atom).all(axis=1))[0][0]
            proj[idx] = hist[i]
        projected_hists.append(proj)

    projected_hists = np.array(projected_hists).T  # (n_bins, n_distributions)

    # Recalculer la cost matrix sur le support commun
    cost_matrix = ot.dist(combined_support, combined_support)

    # Calcul du barycentre régularisé
    bary = ot.bregman.barycenter(projected_hists, cost_matrix, reg)

    # Stocker le barycentre et son support
    bary_flat.append(bary)
    bary_supports.append(combined_support)

# Reshape barycenters
bary_hists = [b.reshape(shape) for b in bary_flat]



While intuitive, this method can suffer from irregularity and instability in barycenters due to the non‑Euclidean curvature of Wasserstein space, which may lead to poor cluster representation and convergence issues.

In [ ]:
assignments, barycenters = bary_WKMeans(
    data=projected_distributions,
    grid=grid,
    n_clusters=3,
    n_iter=10,
    reg=1e-1,
    random_state=None
)

In [ ]:
for i, b in enumerate(barycenters):
    print(f"--- Barycentre {i} ---")
    print(f"Shape: {b.shape}")
    print(f"Min: {b.min():.2e}")
    print(f"Max: {b.max():.2e}")
    print(f"Sum: {b.sum():.6f}")
    print(f"Contains NaN: {np.isnan(b).any()}")
    print(f"Zero elements: {np.sum(b == 0)} / {b.size}")
    print()


In [ ]:
n_bins_age = 20
n_bins_income = 20
extent = [grid[:, 1].min(), grid[:, 1].max(), grid[:, 0].min(), grid[:, 0].max()]

for i, barycenter in enumerate(barycenters):
    barycenter = barycenter.reshape(n_bins_age, n_bins_income)
    
    plt.figure(figsize=(6, 5))
    plt.imshow(barycenter, origin='lower', cmap='viridis', extent=extent, aspect='auto')
    plt.colorbar(label="Probability")
    plt.title(f"Barycentre {i+1}")
    plt.xlabel("Décile de revenu")
    plt.ylabel("Classe d'âge")
    plt.tight_layout()
    plt.show()


In [ ]:
k_values = range(1, 11)
cost_matrix = ot.dist(grid, grid)

inertias = []
for k_test in tqdm(k_values):
    assignments, barycenters = bary_WKMeans(
        data=projected_distributions,
        grid=grid,
        n_clusters=k_test,
        n_iter=100,
        reg=1e-1,
        random_state=None)

    inertia = sum(
        ot.emd2(projected_distributions[i], barycenters[assignments[i]], cost_matrix)
        for i in range(len(projected_distributions))
    )
    inertias.append(inertia)

plt.figure(figsize=(8, 5))
plt.plot(k_values, inertias, marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of clusters k")
plt.ylabel("Total within-cluster Wasserstein distance")
plt.grid(True)
plt.show()


## Clustering based on pairwise distances (D-WKM)

In [ ]:
pairwise_france = computeDistanceMatrix(
    data = projected_distributions,
    grid = grid,
    save=True,
    filepath='../data/Dis_mat_france.txt'
)

# pairwise_france = np.loadtxt("../data/Dis_mat_france.txt", dtype=float)


In [ ]:
assignments_france = dist_WKMeans(
    data = projected_distributions[:10],
    grid = grid,
    dist_matrix=pairwise_france,
    n_clusters=3,
    n_iter=40,
    random_state=None
)

np.unique(assignments_france)

In [ ]:
## Only Paris here
iris_paris = [iris for iris in common_iris if iris.startswith("75")]
distributions_paris = np.array([projected_distributions[common_iris.index(iris)] for iris in iris_paris])

In [ ]:
pairwise_paris = computeDistanceMatrix(
    data = projected_distributions,
    grid = grid,
    save=True,
    filepath='../data/Dis_mat_paris.txt'
)

# pairwise_france = np.loadtxt("../data/Dis_mat_paris.txt", dtype=float)


In [ ]:
assignments_paris = dist_WKMeans(
    data = distributions_paris,
    grid = grid,
    dist_matrix=pairwise_paris,
    n_clusters=3,
    n_iter=40,
    random_state=None
)

np.unique(assignments_paris)

In [ ]:
# pour 1000 observations (3 clusters, 5it)
# clustering_methods : 2m35.2

In [ ]:
k_values = range(1, 11)

inertias = []
for k_test in tqdm(k_values):
    assignments, barycenters = wbarycenter_clustering(
    data=joint_distributions,  # (n_iris, 7, 9)
    n_clusters=k_test,
    n_iter=5,
    reg=0.1,
    random_state=42
)
    inertia = 0
    for i in range(n_distributions):
        bary = barycenters[assignments[i]]
        inertia += ot.wasserstein_1d(support, support, distributions[i], bary)
    inertias.append(inertia)

plt.figure(figsize=(8, 5))
plt.plot(k_values, inertias, marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of clusters k")
plt.ylabel("Total within-cluster Wasserstein distance")
plt.grid(True)
plt.show()

# Mapping

## France

In [ ]:
gdf_france_raw = gpd.read_file("../data/contours-iris.gpkg")

gdf_france = gdf_france_raw[gdf_france_raw["code_iris"].isin(common_iris)].copy()
gdf_france["code_iris"] = gdf_france["code_iris"].astype(str)


iris_cluster_df = pd.DataFrame({
    "code_iris": common_iris,
    "cluster": assignments_france
})

gdf_france = gdf_france.merge(iris_cluster_df, on="code_iris", how="left")

gdf_france['cluster'] = gdf_france['cluster'].astype('category')



In [ ]:
gdf_france.plot(
    column='cluster',
    cmap='Accent',
    legend=True,
    legend_kwds={'title': 'Cluster'},  # Ajoute un titre clair à la légende
    missing_kwds={'color': 'lightgrey', 'label': 'No data'}
)
plt.title("Wasserstein Clusters - France")
plt.axis('off')
plt.show()


## Paris

In [ ]:
iris_paris = [iris for iris in common_iris if iris.startswith("75")]
assignments_proj = np.array([assignments_france[common_iris.index(iris)] for iris in iris_paris])

gdf_paris = gdf_france_raw[gdf_france_raw["code_iris"].isin(iris_paris)].copy()

iris_cluster_df = pd.DataFrame({
    "code_iris": iris_paris,
    "cluster_proj": assignments_proj,
    "cluster_paris": assignments_paris
})

gdf_paris = gdf_paris.merge(iris_cluster_df, on="code_iris", how="left")

gdf_paris['cluster_proj'] = gdf_paris['cluster_proj'].astype('category')
gdf_paris['cluster_paris'] = gdf_paris['cluster_paris'].astype('category')


### France clustering projected on Paris

In [ ]:
gdf_paris.plot(
    column='cluster_proj',
    cmap='Accent',
    legend=True,
    legend_kwds={'title': 'Cluster'},  # Ajoute un titre clair à la légende
    edgecolor='black',
    linewidth=0.1,
    missing_kwds={'color': 'lightgrey', 'label': 'No data'}
)
plt.title("Wasserstein Clusters - Paris")
plt.axis('off')
plt.show()


### Paris Clustering

In [ ]:
gdf_paris.plot(
    column='cluster_paris',
    cmap='Accent',
    legend=True,
    legend_kwds={'title': 'Cluster'},  # Ajoute un titre clair à la légende
    edgecolor='black',
    linewidth=0.1,
    missing_kwds={'color': 'lightgrey', 'label': 'No data'}
)
plt.title("Wasserstein Clusters - Paris")
plt.axis('off')
plt.show()


In [ ]:
n_clusters = 3
fig, axes = plt.subplots(2, n_clusters, figsize=(5*n_clusters, 8))

# France clustering projected on Paris
for k in range(n_clusters):
    ax = axes[0, k]
    idx = rng.choice(np.where(assignments_proj == k)[0])

    dist = joint_distributions_paris[idx]
    supp = joint_supports_paris[idx]
    age_classes = supp[:, 0]
    income_deciles = supp[:, 1]

    scatter = ax.scatter(
        income_deciles, age_classes,
        c=dist,
        cmap='viridis',
        s=300 * dist,
        edgecolor='k'
    )
    ax.set_title(f"France Clustering - Cluster {k}")
    ax.set_xlabel("Income")
    ax.set_ylabel("Age")
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(True)

# Paris clustering
for k in range(n_clusters):
    ax = axes[1, k]
    idx = rng.choice(np.where(assignments_paris == k)[0])

    dist = joint_distributions_paris[idx]
    supp = joint_supports_paris[idx]
    age_classes = supp[:, 0]
    income_deciles = supp[:, 1]

    scatter = ax.scatter(
        income_deciles, age_classes,
        c=dist,
        cmap='viridis',
        s=300 * dist,
        edgecolor='k'
    )
    ax.set_title(f"Paris Clustering - Cluster {k}")
    ax.set_xlabel("Income")
    ax.set_ylabel("Age")
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(True)

fig.tight_layout()
cbar = fig.colorbar(scatter, ax=axes.ravel().tolist(), shrink=0.95)
cbar.set_label("Probabilité")
plt.show()